**Contexto do dataset**

Assim como o famoso 'vinho do porto', Portugal também é conhecido pela produção do 'vinho verde' que é cultivado na região homônima ao noroeste do país entre os rios Douro e Minho. Por constituir dominiação de origem controlada torna-se único ao redor do mundo, apreciado pela leveza e frescor.

O dataset é constituído por 1.599 observações e suas respectivas classificações sensoriais e psicoquímicas. Por questões de privacidade nenhum tipo de uva foi discriminada, assim como rótulos e preços. Temos disponíveis 12 variáveis distintas, sendo que 11 são características sensoriais/psicoquímicas e 1 com o score final do produto (escala de 0 - 10).

Sumário de variáveis:

1 - fixed acidity

2 - volatile acidity

3 - citric acid

4 - residual sugar

5 - chlorides

6 - free sulfur dioxide

7 - total sulfur dioxide

8 - density

9 - pH

10 - sulphates

11 - alcohol

12 - quality (score between 0 and 10)

O dataset foi encontrado no repositório de machine learning da UCI: https://archive.ics.uci.edu/ml/datasets/wine+quality

Citação: P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis. Modeling wine preferences by data mining from physicochemical properties. In Decision Support Systems, Elsevier, 47(4):547-553, 2009.

**Problema de negócio**

Uma grande importadora de vinhos brasileira compreende que a ampliação de sua presença no mercado passa pelo aumento do portifólio de produtos. Tal constatação parte do crescimento da sua maior concorrente (evino.com), que além de ter sido concebida em ambiente digital, possui grande capacidade de negociação e amplo mix de produtos/vinículas.

Com intuíto de ampliar e buscar diferenciação, a importadora se compromete a visitar e negociar com novos produtores. A partir do momento em que começa a prospecção acaba por ter acesso a um universo muito grande de rótulos. Considerando o momento de pandemia, que impossibilita a visitação, e a necessidade imediata, opta por estabelecer uma relação com os vinhos que já fazem parte da sua carta considerando suas características químicas e sensoriais.

O objetivo é encontrar uma relação entre os componentes dos vinhos e suas respectivas notas, habilitando o comprador da importadora a ter uma pré-seleção com base no score atribuído.

Entende-se como 'bom' vinho um score igual ou maior do que 6.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("../input/red-wine-quality-cortez-et-al-2009/winequality-red.csv")
df

In [ ]:
conditions = [
    (df['quality'] >=6),
    (df['quality'] <6)
]

In [ ]:
values = ['1','0']

In [ ]:
df['qualitygood'] = np.select(conditions,values)
df

**Análise exploratória de dados**

Considerações:
* O dataset não possui nenhum null value;
* Por outro lado, diversos outliers foram encontrados entre as variáveis, principalmente nas variáveis 'açucar resídual' e 'cloreto';
* 'Acidez fixa': não é capaz de predizer a qualidade, uma vez que a variação entre diferentes notas foi parecida;
* 'Acidez volátil': indica correlação entre indicador de acidez e nota. Quanto maior nota menor a acidez;
* 'Acidez cítrica': não indica maior ou melhor qualidade, porém as notas mais altas (7 e 8) apontam maior equilíbrio entre as distribuições;
* 'Açucar resídual': variação equilibrada mas com diversos outliers (especialmente com as notas intermediárias como 5 e 6);
* 'Cloreto': variação equilibrada com diversos outliers (especialmente com as notas intermediárias como 5 e 6);
* 'PH': notas mais altas (acima de 6) apresentaram limites inferiores menores do que as demais notas;
* 'Sulfato': notas mais altas apresentaram maior índice de sulfato;
* 'Álcool': notas mais altas apresentaram maior volume de álcool.

In [ ]:
import seaborn as sns

In [ ]:
df.isnull()

In [ ]:
sns.boxplot(data=df, x = 'quality', y = 'fixed acidity') 

In [ ]:
sns.boxplot(data=df, x = 'quality', y = 'volatile acidity')

In [ ]:
sns.boxplot(data=df, x = 'quality', y = 'citric acid') 

In [ ]:
sns.boxplot(data=df, x = 'quality', y = 'residual sugar')

In [ ]:
sns.boxplot(data=df, x = 'quality', y = 'chlorides')

In [ ]:
sns.boxplot(data=df, x = 'quality', y = 'free sulfur dioxide') 

In [ ]:
sns.boxplot(data=df, x = 'quality', y = 'total sulfur dioxide') 

In [ ]:
sns.boxplot(data=df, x = 'quality', y = 'density') 

In [ ]:
sns.boxplot(data=df, x = 'quality', y = 'pH') 

In [ ]:
sns.boxplot(data=df, x = 'quality', y ='sulphates')

In [ ]:
sns.boxplot(data=df, x = 'quality', y ='alcohol') 

In [ ]:
df.plot.scatter(x='quality', y='alcohol')

In [ ]:
df.plot.scatter(x='quality', y='sulphates')

In [ ]:
df.plot.scatter(x='quality', y='density')

In [ ]:
df.plot.scatter(x='quality', y='volatile acidity')

In [ ]:
df.plot.scatter(x='quality', y='citric acid')

**Definição de X e y**

Optou-se por testar o modelo em 30% da amostra.

In [ ]:
X = df.drop(['quality','qualitygood'], axis = 1)
y = df.qualitygood

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

**Treinamento e predição com pipeline**

Dentro do pipeline foi utilizado o 'standarscaler' como padronização e o 'RandomForest' como algortimo de classificação. Abaixo os resultados:

**Matriz de Confusão:**

* Verdadeiro negativo: 155;
* Falso positivo: 58;
* Falso negativo: 71;
* Verdadeiro positivo: 193.

Acurácia: 0.725 Precisão: 0.768 Recal: 0.731

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn import metrics

In [ ]:
pipe = Pipeline([
    ('StandScaler', StandardScaler()),
    ('rf' , RandomForestClassifier(n_estimators=100, max_depth = 4, min_samples_split = 0.1))
])

In [ ]:
pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)

In [ ]:
print(accuracy_score(y_test,y_pred))

In [ ]:
print(confusion_matrix(y_test,y_pred))

In [ ]:
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap = 'coolwarm')

In [ ]:
print(classification_report(y_test, y_pred))

**Treinamento e predição com pipeline e GridSearch**

A utilização do GridSearch aumentou a capacidade de classificação em quase 10 p.p. de acurácia. Abaixo resultados:

**Matriz de Confusão:**

Verdadeiro negativo: 165;
Falso positivo: 48;
Falso negativo: 46;
Verdadeiro positivo: 221.
Acurácia: 0.80

Precisão: 0.821

Recal: 0.827

Cross validation - 5 Kfolds: array([0.69375 , 0.753125 , 0.7125 , 0.759375 , 0.71159875]

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import Lasso

In [ ]:
kf = KFold(n_splits = 3, shuffle = True)

In [ ]:
pipe3 = Pipeline([
    ('minmaxscaler', StandardScaler()),
    ('rf' , RandomForestClassifier())
])

In [ ]:
parameters = {#'selectkbest__k':[8, 12],
              'rf__n_estimators': [200, 300],
              'rf__max_depth': [12, 15],
              'rf__min_samples_split' : [0.0005, 0.0001]}

In [ ]:
gs = GridSearchCV(pipe, 
                  scoring = 'neg_mean_squared_error',
                  param_grid = parameters, 
                  cv = kf,
                  verbose = 1,
                  n_jobs = -1,
                 )

In [ ]:
gs.fit(X_train, y_train)

In [ ]:
gs.cv_results_

In [ ]:
gs.best_params_

In [ ]:
y_pred3 = gs.predict(X_test)

In [ ]:
print(confusion_matrix(y_test,y_pred3))

In [ ]:
sns.heatmap(confusion_matrix(y_test, y_pred3), annot=True, fmt='d', cmap = 'coolwarm')

In [ ]:
print(classification_report(y_test, y_pred3))

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
cross_val_score(RandomForestClassifier(), X, y, cv = KFold(5))

**Considerações finais**

Considerando o cutoff de qualidade como 6 (>=6 é considerado como 'bom'), o modelo possui acuracidade de 80% partindo das características químicas e sensoriais. O algoritimo pode servir de parâmetro para a área comercial, estabelecendo um primeiro filtro em relação à qualidade do vinho.

Como próximos passos a empresa poderia (partindo da sua própria base de dados) executar um algoritmo de regressão com intuíto de atribuir um score inicial, assim como possível precificiação. Parâmetros que devem ser refinados posteriormente com a degustação dos especialistas e aceitação do mercado